# Machine learning for NLP - Project 1

## Libraries and global variables

In [1]:
import numpy as np

In [2]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [3]:
from BM25 import loadNFCorpus, text2TokenList, run_bm25_only

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tboug\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tboug\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ndcg bm25= 0.8135524489389909


## Doc2vec

In [4]:
# documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]

# model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

## Tests

In [5]:
# run_bm25_only(0,150)

ndcg bm25= 0.8135524489389909


0.8135524489389909

In [6]:
# run_bm25_only(0,3192)

ndcg bm25= 0.46335714418774215


0.46335714418774215

In [1]:
from gensim.models import word2vec

In [2]:
import preprocessing as prepro
import models
from metrics import ndcgAt5

In [3]:
dicDoc, dicReq, dicReqDoc = prepro.load_data()
corpus = prepro.build_corpus(dicDoc)
print(corpus[0])

['alkylphenols', 'human', 'milk', 'relations', 'dietary', 'habits', 'central', 'taiwan', 'pubmed', 'ncbi', 'abstract', 'aims', 'study', 'determine', 'concentrations', 'num', 'nonylphenol', 'num', 'octylphenol', 'num', 'human', 'milk', 'samples', 'examine', 'related', 'factors', 'including', 'mothers', 'demographics', 'dietary', 'habits', 'women', 'consumed', 'median', 'amount', 'cooking', 'oil', 'significantly', 'higher', 'concentrations', 'num', 'ng/g', 'consumed', 'num', 'ng/g', 'num', 'concentration', 'significantly', 'consumption', 'cooking', 'oil', 'beta', 'num', 'num', 'fish', 'oil', 'capsules', 'beta', 'num', 'num', 'adjustment', 'age', 'body', 'mass', 'index', 'bmi', 'concentration', 'significantly', 'consumption', 'fish', 'oil', 'capsules', 'beta', 'num', 'num', 'processed', 'fish', 'products', 'beta', 'num', 'num', 'food', 'pattern', 'cooking', 'oil', 'processed', 'meat', 'products', 'factor', 'analysis', 'strongly', 'concentration', 'human', 'milk', 'num', 'determinations', 

In [4]:
docsToKeep,reqsToKeep,dicTrueResults,corpusDocTokenList,corpusDocTokenDic,corpusDocName,corpusDicoDocName,corpusReqName,corpusReqTokenList,corpusReqTokenDic,corpusDicoReqName = prepro.tokenize(dicDoc,dicReq,dicReqDoc)

In [5]:
# w2v = models.model_word2vec(corpusDocTokenDic,corpusReqTokenDic,corpusDocTokenList)

In [6]:
bm25_pred = models.model_bm25(corpusDocTokenList,corpusReqTokenDic)

In [7]:
# w2v_pred = models.model_word2vec_scores_to_list(w2v,corpusDicoDocName)
# ndcgAt5(w2v_pred,dicTrueResults,corpusDicoDocName)

In [8]:
ndcgAt5(bm25_pred,dicTrueResults,corpusDicoDocName)

0.4119320534082562

In [9]:
# bm25_pred_top = models.top_results(bm25_pred,corpusDicoDocName) # Top 10 for the moment
# combination_pred = models.model_word2vec_second(bm25_pred_top,corpusDicoDocName,corpusDocTokenDic,corpus)
# ndcgAt5(combination_pred,dicTrueResults,corpusDicoDocName)

In [10]:
bm25_pred_top = models.top_results(bm25_pred,corpusDicoDocName) # Top 10 for the moment
combination_pred = models.model_bioModel_second(bm25_pred_top,corpusDicoDocName,corpusDocTokenDic,corpus)
ndcgAt5(combination_pred,dicTrueResults,corpusDicoDocName)

0.010332556191092048

In [12]:
import numpy as np

results = bm25_pred_top

model_wordScale = models.model_load_bioModel()

model = {}

# Dictionnary with inverted keys and values of corpusDicoDocName
doc_from_position = {}
for k,v in corpusDicoDocName.items():
    doc_from_position[v] = k

# Get the docs with top results in previous model
for req in results:
    positions = []
    docs_to_evaluate = []
    for i in results[req]:
        if i > 0:
            positions.append(list(results[req]).index(i))
            docs_to_evaluate.append(doc_from_position[list(results[req]).index(i)])

    # Vectorize each one of those doc
    docs_to_evaluate_vectorized = []
    for doc in docs_to_evaluate:
        doc_content = corpusDocTokenDic[doc]
        docs_to_evaluate_vectorized.append(models.model_word2vec_textScale(doc_content,model_wordScale,200))

    req_vectorized = models.model_word2vec_textScale(req,model_wordScale)

    docs_evaluated = []
    for doc in docs_to_evaluate_vectorized:
        docs_evaluated.append(models.cosine_similarity(req_vectorized,doc))

    # Re-fill result with 0 for other documents
    res = np.zeros(len(corpusDicoDocName))
    for j in range(len(docs_evaluated)):
        res[positions[j]] = docs_evaluated[j]

    model[req] = res

In [15]:
set(bm25_pred_top['PLAIN-1'])

{0.0,
 603.2069307608075,
 615.0656401893605,
 615.4491765266109,
 635.2094864381188,
 692.7932885492166,
 790.0560600475912,
 812.0834757801621,
 901.2513516640765,
 947.4604122579915,
 1258.4378073037105}